In [2]:
import pandas as pd
df = pd.read_csv("Labeled/VaxMisinfoData.csv")
from datetime import timedelta
from ratelimit import limits
import requests

import requests
import json
import time
# importing module
import logging
 
# Create and configure logger
logging.basicConfig(filename="antivax.log", format='%(asctime)s %(message)s',filemode='w')
 
# Creating an object
logger = logging.getLogger()
 
# Setting the threshold of logger to DEBUG
logger.setLevel(logging.DEBUG)
 
# Replace your own bearer token below from the academic research project in the Twitter developer portal
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAHrjXQEAAAAAzi7xGphFmwHPwMZTyWzCLH2xW9I%3DVr875clIIOT0fBXeXofXN6yvWb9aXG8HnJvLTKA07ho3NVOVDo'

search_url = "https://api.twitter.com/2/tweets/search/all"

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params):
    response = requests.request("GET",
                                search_url,
                                headers=headers,
                                params=params)
    
    if response.status_code == 429:
        a = int(response.headers["Retry-After"])
        print("Waiting for {} seconds".format(a))
        time.sleep(a)
        response = requests.request("GET", search_url, headers=headers, params=params)
        
    if response.status_code != 200:
        print(response.status_code)
        raise Exception(response.status_code, response.text)
    return response.json()

def main(conversation_id):
        headers = create_headers(bearer_token)
        # Replace with conversation ID below (Tweet ID of the root Tweet)
        query_params = {
            'query': 'conversation_id:{}'.format(conversation_id),
            'tweet.fields': 'id,text,author_id,created_at,lang,public_metrics,in_reply_to_user_id',
            'start_time': '2007-01-25T00:00:00Z',
            'max_results': 500}
        json_response = connect_to_endpoint(search_url, headers, query_params)
        return json_response

In [3]:
df

,id,is_misinfo
0,1344795424855642112,0
1,1344794858133860353,0
2,1344794822691983360,0
3,1344794752819077123,1
4,1344792070507134977,0
...,...,...
15068,1413087751474397186,0
15069,1413087030578401283,0
15070,1413085793397186565,1
15071,1413085519710363648,0


In [4]:
import os
no_exist = []
total = df.shape[0]
so_far = 0
results=[]
for i, j in df.iterrows():
    try:
        t = main(j['id'])
        count = t['meta']['result_count']
        so_far+=count
        if count > 0:
            filename = "comments/{}.json".format(j['id'])
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            with open(filename, "w") as outfile:
                json.dump(t, outfile)
            results.append(count)
            print("Count", count)
    except:
        no_exist.append(i)
    finally:
        logger.info("Done {}/{}".format(i,total))
    time.sleep(3.1)

Count 1
Count 1
Count 2
Count 2
Count 1
Count 4
Count 5
Count 7
Count 23
Count 2
Count 9
Count 18
Count 6
Count 4
Count 7
Count 1
Count 1
Count 5
Count 10
Count 1


KeyboardInterrupt: 

In [9]:
import numpy as np

In [29]:
sum((np.array(results)>5).astype(int))

237

In [230]:
lis = []
import os
import json
for root, dirs, files in os.walk('/Users/varishgrover/Desktop/College/Projects/BTP/twitter/twitter16/comments'):
    for name in files:
        if name.endswith((".json")):
            full_path = os.path.join(root, name)
            lis.append(full_path)

In [231]:
len(lis)

363

In [232]:
import json

In [233]:
counts = []
idx = []
for i in lis:
    f = open(i)
    data = json.load(f)
    idx.append(i[i.rfind('/')+1:i.rfind('.')])
    counts.append(data['meta']['result_count'])

In [234]:
counts = np.array(counts)

In [235]:
import pandas as pd
df = pd.read_csv("/Users/varishgrover/Desktop/College/Projects/BTP/twitter/twitter16/twitter16.csv")

In [236]:
df.head()

,tweetID,source_tweet,label,tree
0,656955120626880512,correct predictions in back to the future ii URL,False,"{'1942819082': '0.0', '1525890199': '0.33', '2..."
1,615689290706595840,.@whitehouse in rainbow colors for #scotusmarr...,True,"{'44945327': '0.0', '20653155': '1.0', '408004..."
2,613404935003217920,cops bought the alleged church shooter burger ...,False,"{'14511951': '0.0', '730793570': '0.52', '6254..."
3,614467824313106432,god put a rainbow over the white house 🌈 URL,True,"{'605822673': '0.0', '1534391900': '0.57', '26..."
4,622891631293935616,#wakeupamerica🇺🇸 who needs a #gun registry whe...,False,"{'26900055': '0.0', '921942782': '2.97', '3222..."


In [237]:
import numpy as np

In [248]:
a = (np.array(counts)>=5)

In [249]:
sum(a.astype(int))

254

In [250]:
curr = np.array(idx)[a]

In [251]:
curr = np.array(curr).astype(np.int64)

In [252]:
df[df['tweetID'].isin(curr)]['label'].value_counts()

False    156
True      98
Name: label, dtype: int64

In [319]:
f = open("/Users/varishgrover/Desktop/College/Projects/BTP/RumourEval-19/test.json")
data = json.load(f)

In [320]:
arr= [len(i['comments']) for i in data]
labels = [i['label'] for i in data]

In [321]:
arr = np.array(arr)
labels = np.array(labels)

In [322]:
len(arr)

81

In [336]:
a = (np.array(arr)>=5)

In [337]:
unique, counts = np.unique(labels[a], return_counts=True)

In [338]:
unique

array(['fake', 'real', 'unverified'], dtype='<U10')

In [339]:
counts

array([32, 28,  8])

In [327]:
sum(a.astype(int))

81

In [1]:
import tweepy, glob

In [2]:
l = glob.glob("comments/*.json")

In [3]:
CONSUMER_KEY="dkjBj2W7TRH5zYy5cTxaYIQbU"
CONSUMER_SECRET="wWwWBPlNERtL7CkuPWxnYwWNAjPtRkDqFos0pELQDeA10o8TWi"
BEARER_TOKEN="AAAAAAAAAAAAAAAAAAAAAJFlPgEAAAAAj2AGoSiNQemC4Su6ZgqJtpi9sMk%3DVLverbgl6iJm7DCdWZznyrtJJdQX6zGJzCS7jUT5Zsmw2Uvxi8"
ACCESS_TOKEN="1278403366201159680-B2VCs3yBnZ01IwkUa5LaTSZ3f9bvOI"
ACCESS_TOKEN_SECRET="nZ6kVFPmWIdq2HPCJAL38gaZ5gu3KXQNU0NUJiiZYAR1f"

In [5]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [8]:
import json
import time
id, text = [], []
lv = 0
for i in l:
    lv+=1
    try:
        d = json.load(open(i))
        tweet = api.get_status(i[i.rfind('/')+1:i.rfind(".")])
        text.append(tweet.text)
        id.append(i)
        print(lv, "Done")
    except:
        print(lv, "ERROR")

1 Done
2 Done
3 ERROR
4 Done
5 ERROR
6 Done
7 Done
8 Done
9 Done
10 ERROR
11 Done
12 Done
13 Done
14 ERROR
15 Done
16 Done
17 Done
18 ERROR
19 Done
20 ERROR
21 Done
22 ERROR
23 Done
24 Done
25 Done
26 ERROR
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
38 Done
39 Done
40 Done
41 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 ERROR
49 Done
50 Done
51 Done
52 Done
53 Done
54 Done
55 Done
56 Done
57 Done
58 Done
59 Done
60 Done
61 Done
62 Done
63 Done
64 Done
65 Done
66 Done
67 Done
68 Done
69 Done
70 Done
71 ERROR
72 Done
73 Done
74 ERROR
75 Done
76 Done
77 Done
78 Done
79 Done
80 Done
81 Done
82 Done
83 Done
84 Done
85 Done
86 Done
87 Done
88 Done
89 Done
90 Done
91 Done
92 Done
93 Done
94 Done
95 Done
96 Done
97 Done
98 Done
99 Done
100 Done
101 Done
102 Done
103 Done
104 ERROR
105 Done
106 Done
107 ERROR
108 Done
109 Done
110 Done
111 Done
112 Done
113 Done
114 Done
115 Done
116 Done
117 Done
118 Done
119 Done
120 Done
121 Done
122 Do

Rate limit reached. Sleeping for: 384


902 Done
903 Done
904 Done
905 Done
906 ERROR
907 Done
908 ERROR
909 ERROR
910 Done
911 Done
912 Done
913 Done
914 Done
915 Done
916 Done
917 ERROR
918 Done
919 Done
920 Done
921 Done
922 Done
923 Done
924 ERROR
925 Done
926 Done
927 Done
928 ERROR
929 Done
930 Done
931 ERROR
932 Done
933 ERROR
934 Done
935 Done
936 Done
937 Done
938 Done
939 Done
940 ERROR
941 Done
942 Done
943 Done
944 Done
945 ERROR
946 ERROR
947 Done
948 Done
949 Done
950 Done
951 Done
952 Done
953 Done
954 Done
955 ERROR
956 Done
957 Done
958 Done
959 Done
960 Done
961 Done
962 Done
963 ERROR
964 Done
965 Done
966 Done
967 Done
968 Done
969 ERROR
970 Done
971 Done
972 Done
973 Done
974 Done
975 Done
976 Done
977 Done
978 Done
979 Done
980 ERROR
981 Done
982 Done
983 Done
984 ERROR
985 Done
986 Done
987 ERROR
988 Done
989 Done
990 Done
991 Done
992 Done
993 Done
994 Done
995 Done
996 Done
997 Done
998 Done
999 Done
1000 Done
1001 Done
1002 Done
1003 Done
1004 Done
1005 Done
1006 Done
1007 Done
1008 Done
1009 Done
1

Rate limit reached. Sleeping for: 418


2534 Done
2535 Done
2536 ERROR
2537 Done
2538 Done
2539 Done
2540 Done
2541 Done
2542 Done
2543 Done
2544 Done
2545 Done
2546 ERROR
2547 Done
2548 Done
2549 Done
2550 ERROR
2551 Done
2552 Done
2553 Done
2554 Done
2555 Done
2556 Done
2557 Done
2558 Done
2559 ERROR
2560 Done
2561 Done
2562 Done
2563 Done
2564 Done
2565 ERROR
2566 Done
2567 ERROR
2568 ERROR
2569 Done
2570 Done
2571 Done
2572 Done
2573 Done
2574 Done
2575 Done
2576 Done
2577 Done
2578 Done
2579 Done
2580 Done
2581 Done
2582 ERROR
2583 Done
2584 Done
2585 Done
2586 Done
2587 Done
2588 Done
2589 Done
2590 Done
2591 Done
2592 Done
2593 Done
2594 Done
2595 Done
2596 Done
2597 Done
2598 Done
2599 Done
2600 Done
2601 Done
2602 ERROR
2603 Done
2604 Done
2605 ERROR
2606 Done
2607 Done
2608 Done
2609 Done
2610 Done
2611 Done
2612 ERROR
2613 Done
2614 Done
2615 Done
2616 Done
2617 Done
2618 Done
2619 Done
2620 Done
2621 Done
2622 Done
2623 Done
2624 Done
2625 Done
2626 ERROR
2627 Done
2628 Done
2629 Done
2630 Done
2631 Done
2632 Don

Rate limit reached. Sleeping for: 529


3434 Done
3435 Done
3436 Done
3437 Done
3438 Done
3439 Done
3440 Done
3441 Done
3442 Done
3443 Done
3444 Done
3445 Done
3446 Done
3447 Done
3448 Done
3449 Done
3450 Done
3451 Done
3452 Done
3453 Done
3454 Done
3455 Done
3456 Done
3457 Done
3458 Done
3459 ERROR
3460 Done
3461 Done
3462 Done
3463 Done
3464 ERROR
3465 Done
3466 Done
3467 Done
3468 Done
3469 Done
3470 Done
3471 Done
3472 ERROR
3473 Done
3474 Done
3475 Done
3476 Done
3477 Done
3478 ERROR
3479 ERROR
3480 Done
3481 Done
3482 Done
3483 Done
3484 Done
3485 Done
3486 Done
3487 Done
3488 Done
3489 Done
3490 Done
3491 Done
3492 Done
3493 Done
3494 Done
3495 Done
3496 Done
3497 Done
3498 Done
3499 ERROR
3500 Done
3501 Done
3502 Done
3503 Done
3504 Done
3505 Done
3506 ERROR
3507 Done
3508 ERROR
3509 Done
3510 Done
3511 Done
3512 Done
3513 Done
3514 Done
3515 Done
3516 Done
3517 ERROR
3518 Done
3519 Done
3520 Done
3521 Done
3522 Done
3523 Done
3524 Done
3525 Done
3526 Done
3527 Done
3528 Done
3529 Done
3530 Done
3531 Done
3532 Done
3

In [19]:
len(text)

3797

In [11]:
import pandas as pd

In [21]:
df = pd.DataFrame()

In [22]:
df['idx'] =  [i[i.rfind('/')+1:i.rfind(".")] for i in id]

In [23]:
df['text'] = text

In [26]:
df.to_csv("antivax.csv", index = False)